In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sat Jan 16 16:06:09 2021

@author: M.W.Sailer

This program simulates a randomly generated deep field image and its characteristics based on major parameters 
of the universe measured from observations. This particular script runs the deterministic code as an ensemble. 
6 graphs are produced in this simulation: 

1. Apparent angular size vs z for an object of fixed proper length in a universe undergoing expansion from dark energy. 
2. Apparent angular size vs z for the same object in a universe without dark energy for comparison.
3. The geometric-focused deep field simulation with galaxies represented as ellipses.
4. The geometric-focused deep field simulation with galaxies represented as ellipses with a Guassian Filter.
5. A reference graph with a filled deep field image necessary for the galaxy coverage percentage calculation.
6. A reference graph with an empty deep field image necessary for the galaxy coverage percentage calculation.
7. Plot of the average angles of galaxy orientation to display anisotropy in the x/y direction.

Necessary Modules:
"""
import numpy as np
from scipy.integrate import quad
import math
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import cv2
import scipy.ndimage as ndimage
from matplotlib.transforms import Bbox
import statistics
plt.ioff()

"""
Ensemble set up
"""

#Set the size of the ensemble
EnsembleNumber = 10 #How many sets of values the ensemble will use
RunNumber = 1 #How many times the ensemble will run each set of values (since random ellipse placement and orientation is used, this number should be more than 1. 10 is recommended)

#Hubble constant perturbation range
Hubblemin = 66.9
Hubblemax = 75.12
Hubblelist = np.linspace(Hubblemin,Hubblemax,EnsembleNumber)

#The matter density perturbation range
Omegammin = 0.2586
Omegammax = 0.37
Omegamlist = np.linspace(Omegammin,Omegammax,EnsembleNumber)

#The dark energy density perturbation range
Omegalambdamin = 0.677
Omegalambdamax = 0.744
Omegalambdalist = np.linspace(Omegalambdamin,Omegalambdamax,EnsembleNumber)

#The dark energy equation of state perturbation range
Wmin = -1.033
Wmax = -0.831
Wlist = np.linspace(Wmin,Wmax,EnsembleNumber)

#The level of anisotropy in the y axis perturbation range
AnisotropyRatiomin = 0.85
AnisotropyRatiomax = 1
AnisotropyRatiolist = np.linspace(AnisotropyRatiomin,AnisotropyRatiomax,EnsembleNumber)

#The number of unseen galaxies in the HUDF perturbation range
Unseenmin = 2
Unseenmax = 10
Unseenlist = np.linspace(Unseenmin,Unseenmax,EnsembleNumber)

#The effective radius increase due to increased sensitivity perturbation range
Halflightmin = 100
Halflightmax = 160
Halflightlist = np.linspace(Halflightmin,Halflightmax,EnsembleNumber)

#The maximum redshift measurement in the JWST deep field image
Zmaxmin = 12
Zmaxmax = 15
Zmaxlist = np.linspace(Zmaxmin,Zmaxmax,EnsembleNumber)

#The universe's age when the first stars were created. This is used for some galaxies' IMFs
Starmin = 1e8
Starmax = 2.5e8
Starlist = np.linspace(Starmin,Starmax,EnsembleNumber)

"""
End of ensemble set up
"""

#Create an empty list to save the galaxy coverage percentage calculations
parameterresults = []

#Use an arbitrary parameter to keep track of the ensemble's progress
n =1 

#Run through each set of parameter values
for pn in range(0,EnsembleNumber):

    
    #Run through each set of parameter values several times to capture the variability caused by the randomness used in the simulation
    for run in range(RunNumber):
        
        #Print the progress of the ensemble
        print('Value #' + str(n))
        print('run # '+str(run+1))

        """
        The Control Panel houses the initial conditions
        -----------------------------------------------------------------------------------------------------------------------------------------------------------
        Control Panel----------------------------------------------------------------------------------------------------------------------------------------------
        -----------------------------------------------------------------------------------------------------------------------------------------------------------
        """
        ########Telescope parameters##########
        primaryMirrorArea = 25.2# m^2   25.2 for JWST,   4.52 for HST
        xaxisminutes = 4.4 # Size of x-axis (arcminutes) in simulation. (4.4 for JWST, or 3.1 for HST)
        yaxisminutes = 2.2 # Size of y-axis (arcminutes) in simulation (2.2 for JWST, 3.1 for HST)
        exposure = 1e4 #seconds
        dpi=1724 #For HST: 1010 yields a resolution of 0.05 arcseconds, for JWST: 1724 yields a resolution of 0.031 arcseconds
        ########Telescope parameters##########
        
        
        ########Perturbation parameters#########
        jwst_Filter = 'F277W'   #For HST, choose closest JWST filter
        ageOfStarBirth = Starlist[pn] #The age of the universe at which star formation first began (years)
        omegamass = Omegamlist[pn] #Baryonic and dark matter mass density
        omegalambda = Omegalambdalist[pn] #Dark energy density
        w = Wlist[pn] #Equation of state parameter of dark energy (between -1/3 and -5/3), typically measured to be ~-1
        hubble = Hubblelist[pn] #hubble Constant in km/s/Mpc
        de_Ratio = AnisotropyRatiolist[pn] # 1 means isotropic expansion, 0.9 equates to 90% of expansion (0.9*H_0) in the y-direction, etc.
        n_Unseen = Unseenlist[pn] # How many times more galaxies exist than are seen in UDF: estimated to be between 2 to 10
        r_Increase = Halflightlist[pn] # % of a galaxy's radius seen due to higher sensitivity of telescope: 100% means anything beyond the halflight radius is invisible, 160 means the visible radius is 1.6 times larger than the effective radius in Allen et al. (2017)'s equation
        z_Max = Zmaxlist[pn] #The maximum measurable redshift reached by the telescope (11 for HUDF, 15 for JWST)
        ########Perturbation parameters##########
        
        
        ########All other parameters##########
        c = 2.9979*10**5 #Speed of light in km/s
        h = 6.626e-34 #(J*s)
        k_Constant = 1.381e-23 #(J/ k)
        extraGalacticBackgroundLight = 9e-9 #Watts
        z_Max_HUDF = 11 #The maximum measured redshifts in the HUDF image (z = 11)
        if xaxisminutes == 4.4:
            resolution = 0.031 #JWST
        else:
            resolution = 0.05 #HST
        ########All other parameters##########
        
        #Initial values for first 2 images. These do not impact the deep field simulation.
        length = 0.025 # proper length (Mpc) used for first two graphs
        z_Value = 20 #Maximum redshift in x-axis of first two graphs
        
        """
        -----------------------------------------------------------------------------------------------------------------------------------------------------------
        End of Control Panel---------------------------------------------------------------------------------------------------------------------------------------
        -----------------------------------------------------------------------------------------------------------------------------------------------------------
        """
        """
        Functions:
        """
        
        #The function 1/(E(z))
        def InverseDensities(z): #Calculates 1/E(z) from z
            return 1/(((1-omegatotal)*(1+z)**2 + omegamass*(1+z)**3 + omegalambda*(1+z)**m)**0.5)
        
        def MpcsToKm(x): #Converts Mpcs to km
            x = x*3.086*10**19 #km
            return x #km
        
        def YearsToSeconds(x): #Converts years to seconds
            x = x*3.154*10**7 #seconds
            return x #seconds
        
        def AgeUniverse(z): #Integration to find the universe's age in years at a given redshift z
            return (MpcsToKm(1)/YearsToSeconds(hubble))/((((1-omegatotal)*(1+z)**2 + omegamass*(1+z)**3 + omegalambda*(1+z)**m)**0.5)*(1+z)) #years
        
        #Function of SAHNI, V., & STAROBINSKY, A. (2000)'s equations.
        def ApparentAngularDiameter(length,z): #Calculates apparanet angular diameter from proper length and redshift
            if k == 0:
                ans, err = quad(InverseDensities, 0, z)
                theta = length*(1+z)**2/((1+z)*c*(1/hubble)*ans)
            elif k == 1:
                ans, err = quad(InverseDensities, 0, z)
                theta = length*(1+z)**2*(abs(omegatotal-1)**0.5)/((1+z)*c*(1/hubble)*math.sin((abs(omegatotal-1)**0.5)*ans))
            else:
                ans, err = quad(InverseDensities, 0, z)
                theta = length*(1+z)**2*(abs(omegatotal-1)**0.5)/((1+z)*c*(1/hubble)*math.sinh((abs(omegatotal-1)**0.5)*ans))
            theta = theta*(180/math.pi)*60 #Arcminutes
            return theta
        
        #Comoving Volume function from Hogg (2000)'s equations.
        def ComovingVolume(z): #Calculates comoving volume from z
            if omegatotal == 1:
                Dh = c/hubble
                ans, err = quad(InverseDensities, 0, z)
                Dc = Dh * ans
                Dm = Dc
                Vc = (4/3)*math.pi*Dm**3
            elif omegatotal > 1:
                Dh = c/hubble
                ans, err = quad(InverseDensities, 0, z)
                Dc = Dh * ans
                Dm = Dh*(1/abs(1-omegatotal)**0.5)*math.sinh(((abs(1-omegatotal))**0.5)*Dc/Dh)
                Vc = ((4*math.pi*Dh**3)/(2*(1-omegatotal)))*((Dm/Dh)*((1+(1-omegatotal)*(Dm/Dh)**2)**0.5)-((1/(abs(1-omegatotal))**0.5)*math.sinh((abs(1-omegatotal))**0.5 * Dm/Dh)))
            else:        
                Dh = c/hubble
                ans, err = quad(InverseDensities, 0, z)
                Dc = Dh * ans
                Dm = Dh*(1/abs(1-omegatotal)**0.5)*math.sin(((abs(1-omegatotal))**0.5)*Dc/Dh)
                Vc = ((4*math.pi*Dh**3)/(2*(1-omegatotal)))*((Dm/Dh)*((1+(1-omegatotal)*(Dm/Dh)**2)**0.5)-((1/(abs(1-omegatotal))**0.5)*math.sin((abs(1-omegatotal))**0.5 * Dm/Dh)))
            return Vc #comoving volume
        
        #find the integration constant "q" using the HUDF z = 11 approximation
        #Integrate q * Comoving Volume * (1+z) from z = 0 to z = 11 and find q that sets integral equal to 10,000
        def GalaxyNumberDensity(z): #Calculates the galaxy number density from comoving volume and galaxy merger estimate (1+z) 
            return (ComovingVolume(z)*(1+z))
        
        #Once q is calculated, the normalized function can be integrated
        def NormalizedGalaxyNumberDensity(z): #normalizes the galaxy number density estimate
            return q*(ComovingVolume(z)*(1+z))
        
        def ExtraGalacticBackgroundLight(watts,area,x,y,seconds,JWSTFilter): #Inputs: EBL power (Watts), area of mirror (m^2), angular size of x-axis of deep field image (arcmin), angular size of y-axis (arcmin), image exposure (s), and JWST filter wavelength
            #This function calculates the number of photons collected by the detector
            B = watts#(Watts/m2/steradian)
            B = B*area #Watts/steradian
            angularAreaArcmin = x*y #arcmin^2
            angularArea = angularAreaArcmin/3600 #square degrees
            steradians = angularArea/ ((180/math.pi)**2) #steradians
            B = B*steradians #Watts
            B = B*seconds #(Joules)
            Energy = h*c/(JWSTFilter/10**9) #Calculates energy (J) per photon
            photons = B/Energy
            photons = photons/((x*60/resolution)*(y*60/resolution)) #photons per pixel
            return photons
        
        def SquareMetersToSteradians(area,z): #meters^2 and redshift
            #This function converts an area in square meters at a redshift z into steradians
            distance = (UniverseAge - (UniverseAge/(1+z)))*9.461e15 #redshift to meters
            radius = (area/(math.pi))**0.5 #radius from area (meters)
            radialangle = np.arctan(radius/distance) #radius in radians
            radialangledegrees = radialangle*180/math.pi #radius in degrees
            angulararea = math.pi*(radialangledegrees**2) #square degrees
            steradians = angulararea/ ((180/math.pi)**2) #steradians
            return steradians
        
        def Blueshift(l,z): #wavelength and redshift
            #This function calculates a new wavelength (blue shift) due to distance z
            l = l/(z+1)
            return l

        def Redshift(l,z): #wavelength and redshift
            #This function calculates a new wavelength (redshift) due to distance z
            l = l*(z+1)
            return l
        
        def BlackBodyGalaxyPhotons(l,T,stars,radius,z,area,exposure):
            #This function calculates the number of photons received by a detector given the following: wavelength (nm), temperature of star (K), stellar radius (m), redshift, area of primary mirror (m^2), and exposure (s)
            z = z #redshift
            l = l/(1*10**9) #nm to meters
            l = Blueshift(l,z)
            R = 0 #starting point to calculate total spectral radiance
            for value in range(len(stars)):
                tempR = stars[value]*(((2*h*c**2)/(l**5))/(np.exp(h*c/(l*k_Constant*T[value]))-1)) # add up all weighted blackbody spectral radiances
                tempR = tempR* math.pi*(radius[value])**2 #multiplies by area of source to get Watts/steradian/m
                R += tempR
            R = R * SquareMetersToSteradians(area,z)  #multiply by angular area entering JWST primary mirror to get Watts/m
            l = Redshift(l,z)
            R = R * l #convert to W by multiplying by wavelength
            R = R * exposure #watts to joules by multiplying seconds
            energy = h*c/l #Joules per photon
            R = R/energy # convert to photons
            R = R*(1/2) #half of light blocked by dust in interstellar medium
            R = R/(1+z) #Decrease of flux due to universe expansion
            if l < 9.12e-8: #Lyman Break
                R = 0
            return R #photons
        
        def VegaPhotons21_4seconds(value,l,MirrorArea,x,y,pixelsize): #inputs: multiplier according to JWST documentation, wavelength, primary mirror area, angular size of x-axis of deep field image (arcmin), angular size of y-axis (arcmin), pixelsize (arcsec)
            #This function calculates maximum number of photons each pixel of the detector can hold before reaching saturation
            flux = 43.6 #in k band photons/cm2/s/A   from https://www.astronomy.ohio-state.edu/martini.10/usefuldata.html 
            flux = flux*21.4 #photons/cm2/A
            l = l*10 #nm to Angstroms
            flux = flux*l*(100)**2 #photons/m2
            flux = flux*MirrorArea #photons
            photons = flux*value #photons
            totalphotons = ((x/(pixelsize/60))*(y/(pixelsize/60))) #Calculates total number of photons received in the deep field image by dividing dimensions by pixel size and multiplying into area
            photons = photons / totalphotons #calculates max number of photons per pixel
            if xaxisminutes == 4.4:
                pass
            else:
                photons = photons*100 #If HUDF is being simulated, decrease sensitivity by ~100
            return photons
        
        def GalaxyStellarDistribution(number,largestMass): #Number of stars in galaxy, and mass of largest stars in galaxy IMF
            totalNumberStars = number
            m=np.linspace(0.01,round(0.01+round(10*largestMass,2)/10,2),1+int(round((round(10*largestMass,2)/10)/0.01,0))) #0.1 solar masses to max solar masses in intervals of 0.1
            normalize = 0
            for mass in m:
                if mass >= 0.5:
                    normalize += (mass)**-2.3
                elif mass > 0.08 and mass < 0.5:
                    normalize += mass**-1.3            *2#IMF smoothing factor
                else:
                    normalize += (mass)**-0.3            *22 #IMF smoothing factor
            starT = []
            starNum = []
            starRadius = []
            
            for mass in m:
                if mass >= 0.5:
                    starNum.append(int((1/normalize)*totalNumberStars*(mass)**-2.3))
                elif mass > 0.08 and mass < 0.5:
                    starNum.append(int((1/normalize)*2*totalNumberStars*(mass)**-1.3))
                else:
                    starNum.append(int((1/normalize)*22*totalNumberStars*(mass)**-0.3))
                if mass >=3:
                    starRadius.append(696e6*mass**(15/19)) #using homology ratios based on sun
                if mass <3:
                    starRadius.append(696e6*mass**(3/7)) #using homology ratios based on sun
                starT.append(5778*mass**(5/8))#relative to the temperature of the sun (K)
            GalaxyParameters = [starT,starNum,starRadius]
            return GalaxyParameters
        
        def GalaxyPick(z): #This function randomly chooses a galaxy type to simulate. Dwarf galaxies are simulated as 50% probability increasing exponentially as z approaches infinity. Non-dwarf galaxies are split into 60% spiral, 15% elliptical, 20% lenticular, and 5% irregular 
            i = (100+100*(1+z))*np.random.rand()
            if i >= 0 and i < 10:
                galaxytype = 'Irregular'
            if i >= 10 and i < 20:
                galaxytype = 'Elliptical'
            if i >=20 and i < 80:
                galaxytype = 'Spiral'
            if i >= 80 and i <100:
                galaxytype = 'Lenticular'
            if i >= 100 and i < 200:
                galaxytype = 'Dwarf_Elliptical'
            if i >= 200:
                galaxytype = 'Dwarf_Spheroidal'
            galaxyType = galaxytype
            ISM = 1.1 + 0.4**np.random.rand() #Estimates the baryonic mass fraction of the interstellar medium as 10-50% (random selection)
            if galaxyType == 'Dwarf_Spheroidal':
                semiMajorLength = 0.0001+(0.0004*np.random.rand())
                semiMinorFraction = 1
                upperMass = 100
                totalMass = 1e7 + (1e8-1e7)*np.random.rand()
                starNumber = totalMass*(1/5)*(1/ISM)*(1/0.3) #4/5 is dark matter, assuming 4/5 of baryonic matter is in the interstellar medium, assuming the average star mass of ~0.3 solar masses 
            if galaxyType == 'Dwarf_Elliptical':
                semiMajorLength = 0.001+(0.009*np.random.rand())
                semiMinorFraction = np.random.rand()
                totalMass = 1e7 + (1e9-1e7)*np.random.rand()
                upperMass = 100
                starNumber = totalMass*(1/5)*(1/ISM)*(1/0.3) #4/5 is dark matter, assuming 4/5 of baryonic matter is in the interstellar medium, assuming the average star mass of ~0.3 solar masses 
            if galaxyType == 'Lenticular':
                semiMajorLength = lengthlist[layer]
                semiMinorFraction = np.random.rand()
                totalMass = 1e8 + (1e14-1e8)*np.random.rand()
                totalMass = totalMass/(1+zlist[layer]) #due to merging
                upperMass = MassUpperLimitNoStarFormation(zlist[layer])
                starNumber = totalMass*(1/5)*(1/ISM)*(1/0.3) #4/5 is dark matter, assuming 0% of baryonic matter is in the interstellar medium, assuming the average star mass of ~0.3 solar masses 

            if galaxyType == 'Spiral':
                semiMajorLength = lengthlist[layer]
                semiMinorFraction = np.random.rand()
                totalMass = 1e9 + (1e14-1e9)*np.random.rand()
                totalMass = totalMass/(1+zlist[layer]) #due to merging
                upperMass = 100
                starNumber = totalMass*(1/5)*(1/ISM)*(1/0.3) #4/5 is dark matter, assuming 4/5 of baryonic matter is in the interstellar medium, assuming the average star mass of ~0.3 solar masses 
            if galaxyType == 'Elliptical':
                semiMajorLength = lengthlist[layer]
                semiMinorFraction = np.random.rand()
                totalMass = 1e8 + (1e14-1e8)*np.random.rand()
                totalMass = totalMass/(1+zlist[layer]) #due to merging
                upperMass = MassUpperLimitNoStarFormation(zlist[layer]) + (100-MassUpperLimitNoStarFormation(zlist[layer]))*np.random.rand()
                starNumber = totalMass*(1/5)*(1/ISM)*(1/0.3) #4/5 is dark matter, assuming 4/5 of baryonic matter is in the interstellar medium, assuming the average star mass of ~0.3 solar masses 
            if galaxyType == 'Irregular':
                semiMajorLength = lengthlist[layer]
                semiMinorFraction = np.random.rand()
                totalMass = 1e8 + (1e10-1e8)*np.random.rand()
                totalMass = totalMass/(1+zlist[layer]) #due to merging
                upperMass = 100
                starNumber = totalMass*(1/5)*(1/ISM)*(1/0.3) #4/5 is dark matter, assuming 4/5 of baryonic matter is in the interstellar medium, assuming the average star mass of ~0.3 solar masses 
            galaxyCharacteristics = [semiMajorLength,semiMinorFraction,totalMass,upperMass,galaxyType,starNumber]
            return galaxyCharacteristics
        
        def MassUpperLimitNoStarFormation(z): #This function calculates the maximum mass in a galaxy's mass function when no star formation occurs after the first stars were created given redshift
            timeSinceBB = UniverseAge/(1+z) #years
            starAge = timeSinceBB-ageOfStarBirth #years
            massUpper = round((starAge/(10**10))**(-1/2.5),2) #mass
            return massUpper #mass
        
        def JWSTBrightSourceLimit(l): #From the JWST doc. this outputs the brightsource limit in Vega mags for each filter
            if l == 'F070W':
                BSL = 14.43
                wavelength = 70
            if l == 'F090W':
                BSL = 15.24
                wavelength = 90
            if l == 'F115W':
                BSL = 15.44
                wavelength = 115
            if l == 'F140M':
                BSL = 14.62
                wavelength = 140
            if l == 'F150W':
                BSL = 15.37
                wavelength = 150
            if l == 'F150W2':
                BSL = 16.60
                wavelength = 150
            if l == 'F162M':
                BSL = 14.40
                wavelength = 162
            if l == 'F164N':
                BSL = 11.99
                wavelength = 164
            if l == 'F182M':
                BSL = 14.36
                wavelength = 182
            if l == 'F187N':
                BSL = 11.68
                wavelength = 187
            if l == 'F200W':
                BSL = 14.80
                wavelength = 200
            if l == 'F210M':
                BSL = 13.66
                wavelength = 210
            if l == 'F212N':
                BSL = 11.38
                wavelength = 212
            if l == 'F250M':
                BSL = 14.14
                wavelength = 250
            if l == 'F277W':
                BSL = 15.23
                wavelength = 277
            if l == 'F300M':
                BSL = 13.93
                wavelength = 300
            if l == 'F322W2':
                BSL = 15.66
                wavelength = 322
            if l == 'F323N':
                BSL = 11.02
                wavelength = 323
            if l == 'F335M':
                BSL = 13.65
                wavelength = 335
            if l == 'F356W':
                BSL = 14.42
                wavelength = 356
            if l == 'F360M':
                BSL = 13.45
                wavelength = 360
            if l == 'F405N':
                BSL = 10.44
                wavelength = 405
            if l == 'F410M':
                BSL = 13.06
                wavelength = 410
            if l == 'F430M':
                BSL = 12.09
                wavelength = 430
            if l == 'F444W':
                BSL = 13.68
                wavelength = 444
            if l == 'F460M':
                BSL = 11.52
                wavelength = 460
            if l == 'F466N':
                BSL = 9.64
                wavelength = 466
            if l == 'F470N':
                BSL = 9.45
                wavelength = 470
            if l == 'F480M':
                BSL = 11.55
                wavelength = 480
            filterparameters = [BSL,wavelength]
            return filterparameters
        
        """
        Preliminary calculations from control panel inputs:
        """
        
        #-----Assigns wavelength and bright source values from filter-------------------------------------------
        wavelength = JWSTBrightSourceLimit(jwst_Filter)[1]
        jwst_Filter_Value = JWSTBrightSourceLimit(jwst_Filter)[0]
        
        #-----Calculation of the total energy density of the universe------------------------------------------
        omegatotal = omegamass+omegalambda
        
        #-----Calculation of the dark energy equation of state-------------------------------------------------
        m = 3*(1+w)
        
        #-----Calculation of the universe's age----------------------------------------------------------------
        age, err = quad(AgeUniverse, 0, float('inf')) 
        
        UniverseAge = age
        
        
        #-----A deep field image is 3-dimensional. Because of this, apparent angular sizes must be calculated for more than
        #a single redshift. This simulation approximates a 3-dimensional universe by generating average galaxies on multiple planes
        #at specific redshifts. To do this, the galaxy density must be known at each redshift. This is a nearly impossible
        #value to find as seen in data from Inami, et. al. (2017) Figure 13 (MUSE-z Distributions can be viewed). This is because
        #telescopes don't detect every galaxy in existence. As distance increases, only the brightest galaxies are detected.
        #Since a reliable equation for galaxy number density based on redshift cannot be obtained, a rough approximation is used until
        #more data is found in future surveys. This approximation assumes galaxy density changes proportional to the Comoving volume X
        #(1+z) to account for expansion and the merger rate estimate according to Conselice et al. (2016).
        
        zlist = [] #start with 2 empty lists
        Densitylist = [] #start with 2 empty lists
        
        #Calculate the distance to the maximum redshift defined as z_Max in Glyrs
        z_MaxGlyrs = (UniverseAge/10**9) - (UniverseAge/10**9)/(z_Max+1)
        
        #Create a list of 24 redshifts to associate a specific plane. This typically corresponds to 0.5 Glyr bins depending on the initial conditions. 
        for plane in list(np.linspace(1,int(z_MaxGlyrs),24)):
            zlist.append(((UniverseAge/10**9)/((UniverseAge/10**9) - plane))-1)
        zlist.append(z_Max)
        
        #-------------Calculating the list of galaxy numbers on each plane--------------------------------------------------
        
        #find the integration constant "q" using the HUDF z = 11 approximation
        #Integrate q * Comoving Volume * (1+z) from z = 0 to z = 11 and find q that sets integral equal to 10,000
        
        ans, err = quad(GalaxyNumberDensity, 0, z_Max_HUDF)
        q = 10000/ans #10000 is used for the calibration according to NASA's HUDF estimate
        
        #Calculate the galaxy number densities on each plane by integrating q * Comoving Volume * (1+z) between each z_Value in zlist
        
        for entry in range(len(zlist)-1): #For each plane
            #Run through same integral between each z value
            
            ans, err = quad(NormalizedGalaxyNumberDensity, zlist[entry], zlist[entry+1])
            Densitylist.append(ans)
            
        #--------------Convert Densitylist into number of galaxies per square arcminute----------------------------------------------------------------
        
        for entry in range(len(Densitylist)):
            if Densitylist[entry]/(3.1*3.1) < 1:
                Densitylist[entry] = 1
            else:    
                Densitylist[entry] = Densitylist[entry]/(3.1*3.1)
                
        #As a calibration, the number of galaxies are multiplied by n_Unseen on each plane less than z = 1.
        #For reference, Conselice et al. (2016) estimates there exist 10 times as many galaxies as can be seen in the HUDF.
        #Lauer et al. (2021) estimates only 2 times as many. No matter which number is chosen, a correction must be made to 
        #account for these unseen galaxies at low redshifts since they should be seen more readily due to the close proximity.  
        #Similarly, the galaxy densities at redshifts should be increased with the JWST (not HST) due to the increased 
        #sensitivity of the JWST. All of this is accounted for below.
        
        #For both HST and JWST simulations, this calibration is made for z<1.
        for z in range(len(zlist)):
            if zlist[z] < 1:
                Densitylist[z] = Densitylist[z] *n_Unseen
            else:
                pass
        
        #For the JWST simulation only, n_Unseen is added for the rest of the galaxies.
        if xaxisminutes == 4.4:
            for density in range(len(Densitylist)):
                if zlist[density] > 1:    
                    Densitylist[density]=Densitylist[density]*n_Unseen #This will automatically change the density to estimated values (only for galaxies further than z = 1)
                else:
                    pass
        else:
            pass
        
        #Average lengths at each redshift plane according to Allen et. Al (2017) r_half-light = 7.07*(1+z)^-0.89 kpc
        #Allen et al's equations are only used at z>1 since galaxies do not appear to grow as much in recent history. 
        #These lengths are then increased up to 160% due to the 100x sensitivity of the JWST according to their Seric profiles.
        lengthlist = []
        for z in range(len(zlist)):
            
            if zlist[z] < 1:
                Averagelength = 7.07*(1+1)**(-0.89) *2 #multiply by 2 to get diameter
                Averagelength = (Averagelength/1000) #Convert from kpcs to Mpcs
            else:
                Averagelength = 7.07*(1+zlist[z])**(-0.89) *2 #multiply by 2 to get diameter
                Averagelength = (Averagelength/1000) # Convert from kpc to Mpcs
            lengthlist.append(Averagelength)
        
        #Definition of k according to the universe's geometry: 1 for critical density, >1 for spherical, <1 for saddle-shaped
        if omegatotal == 1:
            k = 0
        elif omegatotal > 1:
            k = 1
        else:
            k = -1
            
        np.seterr(divide='ignore', invalid='ignore') #The black body spectrum calculations will run into divide errors in python due to tiny numbers being input. This disables the error as this occurs at the extremes and has a negligable impact on the results.
        
        """
        End of preliminary calculations
        """
        
        ####################################################################################################################
        
        #####The Main Calculations:#########################################################################################
        
        ####################################################################################################################
        
        """
        Calculation of apparent size using equation 48 from SAHNI, V., & STAROBINSKY, A. (2000). 
        Equation tested with the sizes and distances of the Bullet Cluster and the Musket Ball Cluster
        as well as a recent paper: Balakrishna Subramani, et al. (2019). This equation has been 
        demonstrated to be accurate for distant objects.
        """
        
        
        """
        FIGURE 1
        Now an apparent size can be calculated using equation 48 from SAHNI, V., & STAROBINSKY, A. (2000). 
        From testing the sizes and distances of the Bullet Cluster and the Musket Ball Cluster as well as 
        a recent paper: Balakrishna Subramani, et al. (2019). This equation has been demonstrated to be 
        accurate for distant objects. Equation 48 changes based on k, so an if statement is used for this 
        calculation. A graph is generated (Figure 1) depicting the apparent angular size of an average 
        galaxy vs Z.
        """
        
        #Create a list of z values from close ~0 to the specified z_Value
        xlist = np.linspace(0.0001,z_Value,10000)
        
        #Create empty ylist
        ylist = []
        
        #fill ylist with values of theta given z and the specified test length
        for i in xlist:
            ylist.append(ApparentAngularDiameter(length,i)) #180/pi converts from radians to degrees, *60 converts degrees to arcminutes
        
        #Plot and save image
        plt.plot(xlist, ylist)
        plt.xlabel('redshift')
        plt.ylabel('angular size (\u0394\u03f4) arcmin')
        plt.title('Apparent Angular Size vs Redshift \nWith Dark Energy \nGalaxy Proper length: '+str(length)+' Mpcs')
        plt.xlim(0,z_Value)
        plt.ylim(0,ylist[9999]*2)
        plt.savefig('fig1.png',dpi=dpi)
        
        #Below prints out the apparent magnitude of the specific z_Value defined above
        i = z_Value
        theta = ApparentAngularDiameter(length,i)
        print('Apparent Magnitude of Test length '+str(length)+' Mpcs at z = '+str(i)+': '+str(round(theta,4)) +' arcminutes')
        
              
        """
        FIGURE 2
        A second plot is generated using trigonometry to plot a graph of apparent angular size vs Z if the universe was not expanding, 
        and the angular size simply decreased with distance (Figure 2). This is created for reference.
        """
        
        #Next Figure
        fig, ax = plt.subplots()
        
        #Create new empty ylist
        newylist = []
        
        #Calculate a theta using basic trigonometry for each z value
        for i in xlist:
            NewDistance = (UniverseAge-(UniverseAge/(i+1)))/(3.262*10**6)#Convert z value to distance in Mpc
            ThetaNoDarkEnergy = (180/math.pi)*np.arctan(length/NewDistance)*60 #180/pi converts from radians to degrees, *60 converts degrees to arcminutes
            newylist.append(ThetaNoDarkEnergy)
        
        #Plot and save image
        plt.plot(xlist, newylist)
        plt.xlabel('redshift')
        plt.ylabel('angular size (\u0394\u03f4) arcmin')
        plt.title('Apparent Angular Size vs Redshift \nWithout Dark Energy \nGalaxy Proper length: '+str(length)+' Mpcs')
        plt.xlim(0,z_Value)
        plt.ylim(0,ylist[9999]*2)
        plt.savefig('fig2.png',dpi=dpi)
        
        """
        FIGURE 3
        Now that estimated apparent angular sizes have been found for each redshift bin, a deep 
        field image can be generated. Galaxies are generated using randomly generated ellipses. 
        The major axis of each ellipse is equal to the apparent angular size at the specified z 
        values. The minor axis is a random value between 0% the apparent angular size and 100% to 
        simulate a random face-orientation of each galaxy since galaxies are seen anywhere from edge-on 
        to face-on. The 1% lower boundary comes from the Milky Way's own proportions. Each galaxy is also 
        randomly oriented at a direction between 0 and 360 degrees since galaxies can be at any angle. 
        
        This will generate Figure 3 (Deep field image)
        """
        
        #A new figure
        #Plot the extragalactic background radiation
        fig, ax = plt.subplots(subplot_kw={'aspect': 'equal'})
        e=Ellipse(xy=(xaxisminutes/2,yaxisminutes/2),width=xaxisminutes*2, height=yaxisminutes*2,angle=0)
        ax.add_artist(e)
        e.set_clip_box(ax.bbox)
        e.set_alpha(ExtraGalacticBackgroundLight(extraGalacticBackgroundLight,primaryMirrorArea,xaxisminutes,yaxisminutes,exposure,wavelength)/int(VegaPhotons21_4seconds(jwst_Filter_Value,wavelength,primaryMirrorArea,yaxisminutes,xaxisminutes,resolution))) 
        e.set_facecolor('k')
        
        #Set up an empty list to save the angle of orientation of every ellipse generated. This will be useful when analyzing the effects of anisotropy.
        EllipseAnglesTotal = []
        DwarfSpheroidal =0
        DwarfElliptical = 0
        Elliptical = 0
        Lenticular = 0
        Spiral =0
        Irregular = 0
        
        #Run through each redshift plane.
        for layer in range(len(Densitylist)):
            
            #Flip the order of layers so furthest layer is generated first. This will allow proper overlapping to occur so high-z galaxies are not generated
            #ontop of low-z galaxies
            layer = len(Densitylist)-1-layer
            #Generate the ellipses:
            #Create empty list of ellipses
            ells=[]
        
            #Run through each x and y grid point
            for galaxy in range(int(xaxisminutes*yaxisminutes*Densitylist[layer])):
                
                galaxyType = GalaxyPick(zlist[layer]) #Choose a galaxy type
                semiMajorLength = galaxyType[0]
                semiMinorFraction = galaxyType[1]
                upperMass = galaxyType[3]
                starNumber = galaxyType[5]
                
                if galaxyType[4] == 'Irregular':
                    Irregular += 1
                if galaxyType[4] == 'Elliptical':
                    Elliptical += 1
                if galaxyType[4] == 'Spiral':
                    Spiral += 1
                if galaxyType[4] == 'Lenticular':
                    Lenticular += 1
                if galaxyType[4] == 'Dwarf_Elliptical':
                    DwarfElliptical += 1
                if galaxyType[4] == 'Dwarf_Spheroidal':
                    DwarfSpheroidal +=1
                
                #Calculation of the apparent angular diameters from SAHNI, V., & STAROBINSKY, A. (2000)'s equations.
                if zlist[layer] <= 1:
                    theta = ApparentAngularDiameter(semiMajorLength,zlist[layer])
                else:
                    theta = (r_Increase/100)*ApparentAngularDiameter(semiMajorLength,zlist[layer])    
            
                #Calculation of the apparent angular diameters from trigonometry if no dark energy existed (for anisotropy incorporation later on)
                NewDistance = (UniverseAge-(UniverseAge/(1+zlist[layer])))/(3.262*10**6)#Convert z value to distance in Mpc
                NoDarkEnergyTheta = (180/math.pi)*np.arctan(lengthlist[layer]/NewDistance)*60 #180/pi converts from radians to degrees, *60 converts degrees to arcminutes
                        
                #Generate a random ellipse orientation angle from 0 to 360 degrees with respect to the x axis
                EllipseAngle = np.random.rand()*360 #360 because input of ellipse is degrees
                EllipseAngleRadians = math.radians(EllipseAngle)#Convert the ellipse angles to radians since python's trig functions require radians instead of degrees
                
                #Calculate the new angle of orientation from the image compression due to anisotropy of dark energy acting on SAHNI, V., & STAROBINSKY, A. (2000)'s equations.
                if EllipseAngleRadians <= math.radians(90): 
                    EllipseAngleNew = math.degrees(math.atan(((NoDarkEnergyTheta*math.sin(EllipseAngleRadians)) + de_Ratio*(theta*math.sin(EllipseAngleRadians) - NoDarkEnergyTheta*math.sin(EllipseAngleRadians)))/(theta*math.cos(EllipseAngleRadians))))
                    EllipseAnglePlot = EllipseAngleNew
                elif EllipseAngleRadians <= math.radians(180):
                    EllipseAngleRadians = math.radians(180)-EllipseAngleRadians
                    EllipseAngleNew = math.degrees(math.atan(((NoDarkEnergyTheta*math.sin(EllipseAngleRadians)) + de_Ratio*(theta*math.sin(EllipseAngleRadians) - NoDarkEnergyTheta*math.sin(EllipseAngleRadians)))/(theta*math.cos(EllipseAngleRadians))))
                    EllipseAnglePlot = 180 - EllipseAngleNew
                elif EllipseAngleRadians <= math.radians(270):
                    EllipseAngleRadians = EllipseAngleRadians-math.radians(180)
                    EllipseAngleNew = math.degrees(math.atan(((NoDarkEnergyTheta*math.sin(EllipseAngleRadians)) + de_Ratio*(theta*math.sin(EllipseAngleRadians) - NoDarkEnergyTheta*math.sin(EllipseAngleRadians)))/(theta*math.cos(EllipseAngleRadians))))
                    EllipseAnglePlot = 180 + EllipseAngleNew
                else:
                    EllipseAngleRadians = math.radians(360)-EllipseAngleRadians
                    EllipseAngleNew = math.degrees(math.atan(((NoDarkEnergyTheta*math.sin(EllipseAngleRadians)) + de_Ratio*(theta*math.sin(EllipseAngleRadians) - NoDarkEnergyTheta*math.sin(EllipseAngleRadians)))/(theta*math.cos(EllipseAngleRadians))))
                    EllipseAnglePlot = 360 - EllipseAngleNew
                
                #Calculate the Ellipse Width:
                #Equal to theta with no anisotropy
                #If anisotropy exists and H0 is less in the y axis than the x axis, the resulting apparent shapes of high-z galaxies will be compressed towards the x axis.
                EllipseWidth = theta*math.cos(EllipseAngleRadians)/math.cos(math.radians(EllipseAngleNew))
                
                #Calculate the Ellipse Height:
                #Similar to Ellipse Width, but with slight difference to equation and random factor from 0 to 1 (simulating random edge-on to top down view)
                    
                EllipseHeight = semiMinorFraction*theta*math.sin(math.radians(EllipseAngleNew))/math.sin(EllipseAngleRadians) 
                
                ellipseAreaPixels = math.pi*EllipseWidth*EllipseHeight/(resolution**2) #number of pixels inwhich galaxy effective radius is detected
                
                GalaxyParameters = GalaxyStellarDistribution(starNumber,upperMass)
                numberPhotons = BlackBodyGalaxyPhotons(wavelength,GalaxyParameters[0],GalaxyParameters[1],GalaxyParameters[2],zlist[layer],primaryMirrorArea,exposure)
                if galaxyType[4] != 'Lenticular' and wavelength*10**-9 < 9.12e-8: #Lyman dropout
                    numberPhotons = 0
                
                alpha = (numberPhotons)/int(VegaPhotons21_4seconds(jwst_Filter_Value,wavelength,primaryMirrorArea,yaxisminutes,xaxisminutes,resolution))
                alpha = alpha/(ellipseAreaPixels) #alpha / number of pixels in the effective radius        
                if alpha > 1: #saturation
                    alpha = 1
                
                #Create ellipse with x,y inputs (adjusted for random distribution), width inputs, height inputs, and angle orientation. Add to ells list.
                ells.append(Ellipse(xy=(xaxisminutes*np.random.rand(),yaxisminutes*np.random.rand()),
                width=EllipseWidth, height=EllipseHeight,
                angle=EllipseAnglePlot))
                
                #Save the ellipse angle
                EllipseAnglesTotal.append(EllipseAnglePlot)
        
            #Plot ellipses
            for e in ells:
                ax.add_artist(e)
                e.set_clip_box(ax.bbox)
                e.set_alpha(alpha)
                e.set_facecolor('k') #Use [1-1/(layer+1),0,1/(layer+1)] so the color of each ellipse is set so that the ellipses change from blue to red depending on the distance. The colors change proportional to 1/distance(Glyrs)
        
        
            #Set axis limits
            ax.set_xlim(0, xaxisminutes)#xaxisminutes)
            ax.set_ylim(0, yaxisminutes)#yaxisminutes)
            
            #Plot axis titles and show layer
            plt.title('A Deep Field Simulation Accounting for the Angular-Diameter-Redshift \nRelation Reaching a Distance of Z='+str(round(z_Max,2))+' with a Total of '
                      +str(len(EllipseAnglesTotal))+' Galaxies')
            plt.xlabel('Arcminutes')
            plt.ylabel('Arcminutes')
            
        
        #Save the figure
        if xaxisminutes == 4.4:
            bbox = Bbox([[0,0.25],[4.4,1.95]]) #for JWST
        else:
            bbox = Bbox([[0.395,0],[2.705,3.1]]) #for HST
        bbox = bbox.transformed(ax.transData).transformed(fig.dpi_scale_trans.inverted())
        plt.savefig('fig3.png',dpi=dpi, bbox_inches=bbox)
        
        
        
        fig, ax = plt.subplots(subplot_kw={'aspect': 'equal'})
        plt.title('A Deep Field Simulation Accounting for the Angular-Diameter-Redshift \nRelation Reaching a Distance of Z='+str(round(z_Max,2))+' with a Total of '
                      +str(len(EllipseAnglesTotal))+' Galaxies')
        plt.xlabel('Arcminutes')
        plt.ylabel('Arcminutes')
        plt.gray()
        img = cv2.imread('fig3.png')
        sigma = 1 #chosen to roughly match the HUDF result
        img = ndimage.gaussian_filter(img, sigma=sigma)
        
        plt.imshow(img, extent=[0,xaxisminutes,0,yaxisminutes])
        plt.savefig('fig3.png', dpi=dpi)
        plt.savefig('fig3-Run_'+str(pn)+'.png', dpi=dpi)
        from google.colab import drive
        drive.mount('/content/drive')
        plt.savefig('/content/drive/My Drive/277/fig3-Run_'+str(pn)+'.png', dpi=dpi)
        
        
        #Calculating the average galaxy angle of orientation in the deep field simulation
        
        #Start with 4 values of zero
        avgright = 0
        numberright = 0
        avgleft = 0
        numberleft = 0
        
        #Analyze each angle and split them into quadrants. Add up every angle in each quadrant and divide by number of angles to determine the average angle.
        for angle in EllipseAnglesTotal:
            if angle<=90:
                avgright+=angle
                numberright+=1
            elif angle>180 and angle<=270:
                avgright+=(angle-180)
                numberright+=1
            elif angle<=180 and angle>90:
                avgleft+=(angle)
                numberleft+=1
            else:
                avgleft+=(angle-180)
                numberleft+=1
        #Calculate the average positively oriented angle (quadrants 1 and 3) and negatively oriented angle (quadrants 2 and 4).
        AverageAngleRight = avgright/numberright
        AverageAngleLeft = avgleft/numberleft
        
        #Print out average positively and negatively oriented angles
        print('Average Positively-Oriented Angle = '+str(round(AverageAngleRight,4)))
        print('Average Negatively-Oriented Angle = '+str(round(AverageAngleLeft,4)))
        
        #Print out the average distance from the 45 degree and 135 degree lines
        print('Anisotropy average distance from 45 or 135 degree angles = '+str(round((abs(45-AverageAngleRight)+abs(45-(AverageAngleLeft-90)))/2,4)))
        
        """
        FIGURES 4 AND 5
        2 Reference Images are created to calculate how much space is covered by the galaxies (Figure 4 and Figure 5)
        """
        
        """
        FIGURE 4: A deep field image completely covered by ellipses
        """
        #A new figure
        fig, ax = plt.subplots(subplot_kw={'aspect': 'equal'})
        
        #Arbitrarily choose a large ellipse diameter to cover the image
        theta = 10
            
        #Arbitrarily choose a grid spacing to display the large ellipses
        Numberx = 5
        Numbery = 5
        x = np.linspace(0,xaxisminutes,Numberx) 
        y = np.linspace(0,yaxisminutes,Numbery) 
        X, Y = np.meshgrid(x, y)
        XY = np.column_stack((X.ravel(), Y.ravel()))
        
        #Create new ellipse list
        ells=[]
        
        #Create the ellipses
        for xx in range(Numberx):#Number):
            for yy in range(Numbery):#Number):
                    EllipseAngle = np.random.rand()*360 #360 because input of ellipse is degrees
                    EllipseAngleRadians = EllipseAngle*2*math.pi/360
                    EllipseWidth = theta
                    EllipseHeight = theta
                    ells.append(Ellipse(xy=(x[xx]+(xaxisminutes/Numberx)*(-0.5+np.random.rand()),y[yy]+(yaxisminutes/Numbery)*(-0.5+np.random.rand())), #adds random factor by changing ellipse location by +/- galaxy radius
                    width=EllipseWidth, height=EllipseHeight,
                    angle=EllipseAngle))
        
        #Make ellipses fully opaque
        for e in ells:
            ax.add_artist(e)
            e.set_clip_box(ax.bbox)
            e.set_alpha(1) #full opaque color = galaxy overlap
            e.set_facecolor('k')
        
        #Plot Ellipses with the exact same title, axes, size, etc. as the deep field image
        ax.set_xlim(0, xaxisminutes)#xaxisminutes)
        ax.set_ylim(0, yaxisminutes)#yaxisminutes)
        plt.title('A Deep Field Simulation Accounting for the Angular-Diameter-Redshift \nRelation Reaching a Distance of Z='+str(round(z_Max,2))+' with a Total of '
          +str(len(EllipseAnglesTotal))+' Galaxies')
        plt.xlabel('Arcminutes')
        plt.ylabel('Arcminutes')
        
        #Save the figure as "Maximum"
        plt.savefig('Maximum.png',dpi=dpi)
        
        """
        FIGURE 5: A deep field image completely empty of ellipses
        """
        #A new figure
        fig, ax = plt.subplots(subplot_kw={'aspect': 'equal'})
        
        #Plot Ellipses with the exact same title, axes, size, etc. as the deep field image
        ax.set_xlim(0, xaxisminutes)#xaxisminutes)
        ax.set_ylim(0, yaxisminutes)#yaxisminutes)
        plt.title('A Deep Field Simulation Accounting for the Angular-Diameter-Redshift \nRelation Reaching a Distance of Z='+str(round(z_Max,2))+' with a Total of '
          +str(len(EllipseAnglesTotal))+' Galaxies')
        plt.xlabel('Arcminutes')
        plt.ylabel('Arcminutes')
        
        #Save the figure as "Minimum"
        plt.savefig('Minimum.png',dpi=dpi)
        
        
        """
        FIGURE 6
        Finally, a 6th image is generated which displays the effects from anisotropy.
        """
        
        #A new figure
        fig, ax = plt.subplots(subplot_kw={'aspect': 'equal'})
        
        #Plot a cartesian plane
        V = np.array([[1,0], [-2,0], [0,-2], [0,1]])
        origin = np.array([[0, 0, 0, 0],[0, 0, 0, 0]]) # origin point
        plt.quiver(*origin, V[:,0], V[:,1], color=['k','k','k','k'], scale=2, width=0.015)
        V = np.array([[1,1], [-1,-1], [1,-1], [-1,1]])
        origin = np.array([[0, 0, 0, 0],[0, 0, 0, 0]]) # origin point
        plt.quiver(*origin, V[:,0], V[:,1], color=['k','k','k','k'], scale=2, linestyle='dashed', width=0.001)
        
        #Plot 2 vectors representing the average positively and negatively oriented galaxy angle 
        V = np.array([[math.cos(AverageAngleRight*2*math.pi/360),math.sin(AverageAngleRight*2*math.pi/360)], [math.cos(AverageAngleLeft*2*math.pi/360),math.sin(AverageAngleLeft*2*math.pi/360)]])
        origin = np.array([[0, 0],[0, 0]]) # origin point
        plt.quiver(*origin, V[:,0], V[:,1], color=['r','b',], scale=2, headwidth=2,width=0.01)
        V = np.array([[-math.cos(AverageAngleRight*2*math.pi/360),-math.sin(AverageAngleRight*2*math.pi/360)], [-math.cos(AverageAngleLeft*2*math.pi/360),-math.sin(AverageAngleLeft*2*math.pi/360)]])
        origin = np.array([[0, 0],[0, 0]]) # origin point
        plt.quiver(*origin, V[:,0], V[:,1], color=['r','b',], scale=2, headwidth=1,width=0.01)
        
        #Format the image
        ax = fig.gca()
        ax.set_xlim(-1, 1)
        ax.set_ylim(-1, 1)
        ax.set_xticks(np.arange(-1, 1.5, 0.5))
        ax.set_yticks(np.arange(-1, 1.5, 0.5))
        ax.set_xlim(-1, 1)
        ax.set_ylim(-1, 1)
        plt.title('The Average Positively-Oriented Angle (Red) \nand Negatively-Oriented Angle (Blue) \nRepresented with Vectors of Magnitude 1')
        plt.xlabel('Total number of galaxies= '+str(len(EllipseAnglesTotal))+'    Positive: '+str(numberright)+'    Negative: '+str(numberleft))
        
        plt.savefig('fig4.png',dpi=dpi)
        
        #Print the average galaxy angle orientation. This should also point along the direction of less expansion indicating an axis for anisotropy
        print('Direction of anisotropy: '+str(round((AverageAngleRight+AverageAngleLeft)/2,4)))
        
        """
        FINAL CALCULATION
        Now a percentage of space taken up by the galaxies can be calculated by importing the 2 reference images and the deep field image.
        """
        
        threshold = 0.1+ExtraGalacticBackgroundLight(extraGalacticBackgroundLight,primaryMirrorArea,xaxisminutes,yaxisminutes,exposure,wavelength)/int(VegaPhotons21_4seconds(jwst_Filter_Value,wavelength,primaryMirrorArea,yaxisminutes,xaxisminutes,resolution)) #fraction at which anything above will be seen as a galaxy #HUDF results in ~4.49% with a value of 0.15
        
        #The previously generated "Maximum" reference image is loaded into python and the number of white pixels is counted
        img = cv2.imread('Maximum.png', cv2.IMREAD_GRAYSCALE)
        n_white_pix_max = np.sum(img >= 255*(1-threshold))
        
        #The previously generated "Minimum" reference image is loaded into python and the number of white pixels is counted
        img = cv2.imread('Minimum.png', cv2.IMREAD_GRAYSCALE)
        n_white_pix_min = np.sum(img >= 255*(1-threshold))
        
        #The white pixel difference is found representing the total number of pixels lying in the deep field simulation image. 
        n_difference = n_white_pix_min-n_white_pix_max
        ntotal = img.size
        
        #The previously generated deep field image is loaded in and the number of white pixels is calculated
        img = cv2.imread('fig3.png', cv2.IMREAD_GRAYSCALE)
        n_white_pix_image = np.sum(img >= 255*(1-threshold))
        
        #The number of white pixels from the "Maximum" deep field image is subtracted from the number of white pixels in the deep field image
        #resulting in the number of white pixels (the pixels not covered by an ellipse). This number is divided by the total possible number 
        #of pixels in the deep field image to find what percentage of the simulation is covered by galaxies. This percentage is displayed as
        #the final calculation. 
        fraction=(n_white_pix_image-n_white_pix_max)/n_difference
        print("Percentage of Wall Formed: "+str(round((1-fraction)*100,2))+'%')
        print('A total of '+str(Irregular)+' Irregular, '+str(Elliptical)+' Elliptical, '+str(Spiral)+' Spiral, '+str(Lenticular)+' Lenticular, '+str(DwarfSpheroidal)+ ' Dwarf Spheroidal, and ' +str(DwarfElliptical)+' Dwarf Elliptical galaxies were simulated.')

        
        #Add the resulting galaxy coverage percent to a list
        parameterresults.append(round((1-fraction)*100,2))
        print(parameterresults)

    #Add 1 to the arbitrary parameter to keep track of the ensemble's progress
    n+=1

#Calculate the mean and standard deviation of galaxy coverage percentages in the list
print('Average Percent Covered by Galaxies: '+str(statistics.mean(parameterresults))+' +/- '+str(statistics.stdev(parameterresults)))

#Display results in an image
fig, ax = plt.subplots(subplot_kw={'aspect': 'equal'})
plt.title('Average Percent Covered by Galaxies:\n'+str(round(statistics.mean(parameterresults),4))+' +/- '+str(round(statistics.stdev(parameterresults),4)))
plt.savefig('Ensemble_Result.png',dpi=dpi)
